Syed Ahmed Haseeb. 2022557. Lab task 2



In [7]:
import heapq  # Importing heapq to implement the priority queue for A* algorithm
import random  # Importing random to place obstacles and malfunctioning robots randomly
import math    # Importing math for distance calculations

# Function to create a grid based on user-defined size and set the charging station at a random position
def create_grid(size):
    grid = [[' ' for _ in range(size)] for _ in range(size)]  # Create an empty grid filled with spaces
    grid[0][0] = 'R'  # 'R' marks the robot starting point at the top-left corner (0,0)

    # Set a random position for the charging station
    charging_station = (random.randint(0, size-1), random.randint(0, size-1))
    grid[charging_station[0]][charging_station[1]] = 'C'

    return grid, charging_station  # Return the grid and charging station's position

# Function to add random obstacles and malfunctioning robots to the grid
def add_obstacles_and_robots(grid, num_obstacles, num_robots):
    size = len(grid)
    for _ in range(num_obstacles):
        obstacle_x, obstacle_y = random.randint(0, size-1), random.randint(0, size-1)
        # Ensure obstacles are not placed on the robot 'R' or charging station 'C'
        while grid[obstacle_x][obstacle_y] in ['R', 'C']:
            obstacle_x, obstacle_y = random.randint(0, size-1), random.randint(0, size-1)
        grid[obstacle_x][obstacle_y] = 'X'  # Place the obstacle 'X'

    for _ in range(num_robots):
        robot_x, robot_y = random.randint(0, size-1), random.randint(0, size-1)
        # Ensure malfunctioning robots are not placed on the robot 'R', charging station 'C', or obstacles 'X'
        while grid[robot_x][robot_y] in ['R', 'C', 'X']:
            robot_x, robot_y = random.randint(0, size-1), random.randint(0, size-1)
        grid[robot_x][robot_y] = 'M'  # Place the malfunctioning robot 'M'

    return grid

# Function to check if a position is valid (within bounds and not blocked by an obstacle or malfunctioning robot)
def is_valid_position(grid, x, y):
    size = len(grid)
    return 0 <= x < size and 0 <= y < size and grid[x][y] not in ['X', 'M']  # Return true if position is valid

# Heuristic function: Calculates the straight-line (Euclidean) distance between the current node and the goal
def heuristic(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def a_star(grid, start, goal):
    size = len(grid)
    # Priority queue to keep track of nodes to explore (using heapq)
    open_list = []
    heapq.heappush(open_list, (0, start))  # Add the start node with priority 0

    # Dictionaries to store the cost to reach each node and the path taken
    g_score = {start: 0}  # Cost from start to current node
    parent = {start: None}  # To reconstruct the path

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Possible directions: up, down, left, right

    while open_list:
        _, current = heapq.heappop(open_list)  # Get the node with the lowest f(n)

        # If we reach the goal (charging station), stop the search
        if current == goal:
            break

        # Explore neighboring positions (up, down, left, right)
        for direction in directions:
            next_x = current[0] + direction[0]  # Calculate next x-coordinate
            next_y = current[1] + direction[1]  # Calculate next y-coordinate
            next_state = (next_x, next_y)  # Form the next state (position)

            # Check if the next position is valid
            if is_valid_position(grid, next_x, next_y):
                tentative_g_score = g_score[current] + 1  # Cost of moving to the next position

                # If the next state has not been explored or we found a cheaper path to it
                if next_state not in g_score or tentative_g_score < g_score[next_state]:
                    g_score[next_state] = tentative_g_score  # Update the cost to reach this state
                    f_score = tentative_g_score + heuristic(next_state, goal)  # Calculate the total cost f(n)
                    heapq.heappush(open_list, (f_score, next_state))  # Add the state to the open list
                    parent[next_state] = current  # Set the current state as the parent of the next state

    # Reconstruct the path from start to goal
    path = []
    current = goal  # Start from the goal and work backwards

    # Check if the goal is in the parent dictionary
    if current in parent:
        while current is not None:
            path.append(current)  # Add current position to the path
            current = parent[current]  # Move to the parent of the current state
        path.reverse()  # Reverse the path to start from the beginning (start to goal)
    else:
        print("No path found to the goal.")
        path = []  # Return an empty path if no path found

    return path

# Function to print the grid with lines and the path marked
def print_grid_with_path(grid, path):
    grid_with_path = [row.copy() for row in grid]  # Copy the original grid to preserve it
    for (x, y) in path:
        if grid_with_path[x][y] not in ['R', 'C']:  # Don't overwrite start 'R' and goal 'C'
            grid_with_path[x][y] = '*'  # Mark the path with '*'

    # Print the grid with lines to separate cells clearly
    print("\nGrid with Path:")
    print('-' * (len(grid_with_path) * 4 + 1))  # Print top border line
    for row in grid_with_path:
        print('| ' + ' | '.join(row) + ' |')  # Print row with borders between cells
        print('-' * (len(grid_with_path) * 4 + 1))  # Print horizontal line after each row

# Main function to play the robot navigation game
def robot_navigation():
    # Ask the user for grid size, number of obstacles, and number of malfunctioning robots
    size = int(input("Enter the grid size (e.g., 6 for a 6x6 grid): "))
    num_obstacles = int(input(f"Enter the number of obstacles (less than {size * size - 2}): "))  # Make sure obstacles are less than available spaces
    num_robots = int(input(f"Enter the number of malfunctioning robots (less than {size * size - 2 - num_obstacles}): "))  # Make sure malfunctioning robots are less than available spaces

    # Create the grid and set the charging station at a random position
    grid, charging_station = create_grid(size)

    # Add random obstacles and malfunctioning robots to the grid
    grid = add_obstacles_and_robots(grid, num_obstacles, num_robots)

    # Print the initial grid
    print("\nInitial Grid:")
    print_grid_with_path(grid, [])  # Empty path initially

    # Start the search using A* algorithm
    print("\nSearching for the optimal path to the charging station using A* algorithm...\n")
    path = a_star(grid, (0, 0), charging_station)  # Perform A* search

    # Print the final grid with the path marked
    print("\nPath to the Charging Station:")
    print_grid_with_path(grid, path)  # Show the path to the charging station

# Run the game
robot_navigation()



Enter the grid size (e.g., 6 for a 6x6 grid): 6
Enter the number of obstacles (less than 34): 3
Enter the number of malfunctioning robots (less than 31): 4

Initial Grid:

Grid with Path:
-------------------------
| R | M | X |   |   |   |
-------------------------
|   |   |   |   | M | M |
-------------------------
|   |   |   |   |   |   |
-------------------------
|   |   |   | C |   |   |
-------------------------
|   |   |   |   |   |   |
-------------------------
|   |   |   | X |   | M |
-------------------------

Searching for the optimal path to the charging station using A* algorithm...


Path to the Charging Station:

Grid with Path:
-------------------------
| R | M | X |   |   |   |
-------------------------
| * | * | * |   | M | M |
-------------------------
|   |   | * | * |   |   |
-------------------------
|   |   |   | C |   |   |
-------------------------
|   |   |   |   |   |   |
-------------------------
|   |   |   | X |   | M |
-------------------------
